# Initialize the model
import gurobipy as gp
from gurobipy import GRB

In [39]:
# Dictionary to store time for each job
job_times = {
    1: 4,
    2: 2,
    3: 4,
    4: 6,
    5: 7,
    6: 5,
    7: 1,
    8: 4,
    9: 3,
}


In [40]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Define the precedence dictionary based on your table (activity -> list of immediate predecessors)
precedence_dict = {
    1: [],
    2: [],
    3: [1],
    4: [1,2],
    5: [3,4],
    6: [4],
    7: [5],
    8: [6],
    9: [7,8],
}

num_jobs = 9

# Initialize an empty precedence matrix with zeros (16x16)
precedence_matrix = np.zeros((num_jobs, num_jobs), dtype=int)

# Fill the precedence matrix based on the precedence_dict
for activity, predecessors in precedence_dict.items():
    for predecessor in predecessors:
        precedence_matrix[activity - 1][predecessor - 1] = 1 

# Print the precedence matrix to verify
print("Precedence Matrix:")
print(precedence_matrix)


Precedence Matrix:
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 1 0]]


In [41]:
# Dictionary to store results for each number of stations
results = {}
max_num_stations = 9

    
# Create a new Gurobi model
model = gp.Model("assembly_line_balancing")

# Create a matrix of Gurobi binary variables X_{is} 
X = [[model.addVar(vtype=GRB.BINARY, name="X_{}_{}".format(i+1, s+1)) 
        for s in range(max_num_stations)] 
        for i in range(num_jobs)]

# Add y variables to indicate if a station is active
Y = [model.addVar(vtype=GRB.BINARY, name="Y_{}".format(s+1)) for s in range(max_num_stations)]

model.setObjective(gp.quicksum(Y), GRB.MINIMIZE)

# Occurance Constraint
for i in range(num_jobs):
    model.addConstr(
        gp.quicksum(X[i][s] for s in range(max_num_stations)) == 1,
        name=f"assignment_job_{i+1}"
    )


# Cycle Time Constraint
for s in range(max_num_stations):
    model.addConstr(
        gp.quicksum(X[i][s] * job_times[i+1] for i in range(num_jobs)) <= 15 * Y[s],
        name=f"cycle_time_station_{s+1}"
    )

# Add precedence constraints to the model
for i in range(num_jobs):
    for j in range(num_jobs):
        if precedence_matrix[i][j] == 1:
            for k in range(max_num_stations):
                lhs = X[i][k]
                rhs = gp.quicksum(X[j][z] for z in range(k+1))
                model.addConstr(lhs <= rhs, name=f"precedence_{i+1}_{j+1}_station_{k+1}")

# Update the model to integrate the new constraints
model.update()

# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 108 rows, 90 columns and 711 nonzeros
Model fingerprint: 0x895af3ae
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 26 rows and 10 columns
Presolve time: 0.00s
Presolved: 82 rows, 80 columns, 425 nonzeros
Variable types: 0 continuous, 80 integer (80 binary)

Root relaxation: objective 2.400000e+00, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.40000    0    3    4.00000    2.40000  40.0% 

In [42]:
# Iterate through each station and print the activities assigned to it
for s in range(max_num_stations):
    if Y[s].X > 0.5:  # Check if the station is used
        activities_in_station = [i+1 for i in range(num_jobs) if X[i][s].X > 0.5]
        print(f"Station {s+1} is used and contains activities: {activities_in_station}")

Station 6 is used and contains activities: [1, 2, 3]
Station 8 is used and contains activities: [4, 6, 8]
Station 9 is used and contains activities: [5, 7, 9]


In [43]:
# Iterate through each station and print the activities assigned to it along with their processing times
for s in range(max_num_stations):
    if Y[s].X > 0.5:  # Check if the station is used
        activities_in_station = [i+1 for i in range(num_jobs) if X[i][s].X > 0.5]
        processing_times = [job_times[activity] for activity in activities_in_station]
        total_processing_time = sum(processing_times)
        print(f"Station {s+1} is used and contains activities: {activities_in_station} with processing times: {processing_times}")
        print(f"Total processing time for station {s+1}: {total_processing_time}")

Station 6 is used and contains activities: [1, 2, 3] with processing times: [4, 2, 4]
Total processing time for station 6: 10
Station 8 is used and contains activities: [4, 6, 8] with processing times: [6, 5, 4]
Total processing time for station 8: 15
Station 9 is used and contains activities: [5, 7, 9] with processing times: [7, 1, 3]
Total processing time for station 9: 11
